In [1]:
using SparseArrays
using LinearAlgebra
using Arpack
import FFTW
# natural units
a=m=hbar=1
# spatial grid: -a/2 < x < a/2
nmax=50
xgrid=a*(range(1,length=nmax)/(nmax+1) .- .5)

ArgumentError: ArgumentError: Package Arpack not found in current path:
- Run `import Pkg; Pkg.add("Arpack")` to install the Arpack package.


In [3]:
# momentum operator
pM=spzeros(Complex{Float64}, nmax, nmax)
for row in 1:nmax
    for col in 1:nmax
        if ((row - col) % 2) == 0
            pM[row,col]=0
        else
            pM[row,col] = 4im*hbar*row*col/(a*(col*col-row*row))
        end
    end
end
# identity operator
idM = sparse(I,nmax,nmax)
# kinetic operator in momentum basis
TM = spdiagm(0=>pi^2*hbar^2/(2*m*a^2)*range(1,length=nmax).^2)
# position operator in the position representation
xP = spdiagm(0=>xgrid)
# position operator in the momentum representation
xM = FFTW.r2r(xP, FFTW.RODFT00)/(2*(nmax+1))
# projection operator in position and momentum representations
PiP(n::Integer) = sparse([n],[n],[1],nmax,nmax)
PiM(n::Integer) = FFTW.r2r(PiP(n), FFTW.RODFT00)/(2*(nmax+1))
# spin 1/2 (todo: use generic spin generators)
sx = sparse([0 .5; .5 0])
sy = sparse([0 -.5; .5 0])
sz = sparse([.5 0; 0 -.5])
idS = sparse([1 0; 0 1])

2×2 SparseMatrixCSC{Int64,Int64} with 2 stored entries:
  [1, 1]  =  1
  [2, 2]  =  1

In [4]:
# Hamiltonian in momentum basis
HM(delta, alpha) = kron(TM, idM, idS) + kron(idM, TM, idS) + delta*kron(idM, idM, sz) + alpha*(kron(pM, idM, sy) - kron(idM, pM, sx))
# ground state
gs(delta, alpha) = eigs(HM(delta, alpha), nev=1, which=:SR)

gs (generic function with 1 method)

In [5]:
gamma=gs(1,20)[2]
#HM(1,20)[1,52]

5000×1 Array{Complex{Float64},2}:
   3.642050773923621e-12 - 1.385951380676529e-12im 
      0.1014863735585993 - 0.0033670900673521778im 
   -0.013419399159065431 - 0.40446977323089917im   
  -5.619461374831026e-12 - 1.4678614860512184e-11im
  -2.265396075489369e-11 + 8.688451679661158e-12im 
     -0.6340707378814023 + 0.021037043778926522im  
    0.018716555617278984 + 0.5641296541264014im    
   7.809991587307295e-12 + 2.032924982641788e-11im 
  1.0177828655670217e-11 - 3.919551998569373e-12im 
     0.28450082960861256 - 0.009439098904979093im  
   -0.001281890756754963 - 0.03863705502356507im   
  -5.418201197951602e-13 - 1.3905894663975096e-12im
  1.3993029997394474e-12 - 5.359432325501265e-13im 
                         ⋮                         
  -2.367748834172592e-11 - 7.136556117585847e-10im 
   5.878090034811293e-18 - 1.5559159335008956e-17im
   3.761506115000529e-18 - 3.121680821580155e-18im 
   1.3778468213122191e-9 - 4.571386665565719e-11im 
 -1.1103856600945522e-11 - 3.3

In [6]:
X = dot(gamma, kron(xM, idM, idS) * gamma)

-9.186428575102775e-14 - 6.058451752097371e-28im

In [7]:
XX=dot(gamma, kron(xM*xM, idM, idS) * gamma)

0.03821209864844788 + 2.168404344971009e-19im

In [9]:
YY=dot(gamma, kron(xM, xM, idS) * gamma)

-0.0006128196231475039 - 1.3552527156068805e-20im

In [ ]:
rdm(psiABC, dA, dB, dC)